# Bagging for Regression Trees



In [ ]:
options(repr.plot.width = 15, repr.plot.height = 10)



## Goal

In this tutorial, we will demonstrate the use of an ensemble method known as *bagging* for
*regression trees*. We will study the `Boston` dataset available in the `R` package `MASS`, which contains information on housing values in the suburbs of Boston with variables such as:

* **medv**: median value of owner-occupied homes in \$1000s (response variable);
* **crim**: per capita crime rate by town;
* **zn**: proportion of residential land zoned for lots over 25,000 sq. ft.;
* **indus**: proportion of non-retail business acres per town;
* **chas**: Charles River dummy variable (1 if tract bounds river; 0 otherwise);
* **nox**: nitrogen oxides concentration (parts per 10 million);
* **rm**: average number of rooms per dwelling;
* **age**: proportion of owner-occupied units built before 1940;
* **dis**: weighted distances to five Boston employment centers;
* **rad**: index of accessibility to radial highways;
* **tax**: full-value property tax rate per \$10,000;
* **ptratio**: pupil-teacher ratio by town;
* **black**: 1000(Bk – 0.63)², where Bk is the proportion of Black residents by town; and
* **lstat**: percentage of lower-status population.

## Load libraries and data

First, we load the required libraries. We use the `MASS` package because the `Boston`
dataset is available there, and the `rpart` package because it contains implementations of
decision trees.


In [ ]:
library(MASS)
library(rpart)
library(ggplot2)
theme_set(theme_gray(base_size = 18))


To load an available dataset from a package, simply use the function `data`.



In [ ]:
data(Boston)
number_of_data_rows <- nrow(Boston)
head(Boston)


## Exploratory analysis

We begin by obtaining summaries of the variables in the Boston dataset.


In [ ]:
summary(Boston)



We will analyze the median value of homes (`medv`):



In [ ]:
ggplot(Boston) +
    geom_histogram(aes(x = medv), bins = 20, color = "black", fill = "lightblue") +
    labs(x = "Median home value", y = "Frequency")


## Resampling the original dataset using bootstrap

In order to apply bagging, we first create a list of resampled datasets.

### Example of obtaining a bootstrapped dataset

A bootstrapped dataset is simply obtained by sampling the original dataset with
replacement. This can be done as follows:


In [ ]:
shuffled_row_numbers <- sample(1:number_of_data_rows, number_of_data_rows, replace = TRUE)
shuffled_row_numbers


In [ ]:
bootstrap_data <- Boston[shuffled_row_numbers, ]
head(bootstrap_data)


### Create a list of bootstrapped datasets

Now, let's repeat the previous procedure 100 times to create resampled datasets. We set a
seed with `set.seed()` to be able to reproduce the same list of resampled datasets:


In [ ]:
set.seed(45632)
bootstrap_list <- list()
number_of_resamples <- 100

for (i in 1:number_of_resamples) {
    shuffled_row_numbers <- sample(1:number_of_data_rows, number_of_data_rows, replace=TRUE)
    bootstrap_list[[i]] <- Boston[shuffled_row_numbers, ]
}

lapply(bootstrap_list[1:5], head)


## Bagging implementation

The idea of bagging is to fit a decision tree algorithm to each bootstrapped dataset.

1. First, we recall how to fit a pruned regression tree, and
2. then, we fit this model to each bootstrapped dataset.

### Pruned regression trees

To fit a pruned regression tree, we can use the `rpart` package as follows. First, let's
fit a decision tree and take a look at the *complexity parameter table*:


In [ ]:
set.seed(123)
aux_fit <- rpart(medv ~ ., data = bootstrap_list[[1]])
aux_fit$cptable


* *CP*: the penalty for each split;
* *nsplit*: the number of splits the tree has at that stage;
* *rel error*: relative error compared to the null model;
* *xerror*: cross-validated relative error (usually 10-fold CV); and
* *xstd*: standard error for the cross-validated error.

Identify the penalty with the lowest cross-validated error:


In [ ]:
aux_id_lowest_error <- which.min(aux_fit$cptable[, "xerror"])
aux_id_lowest_error


Prune the tree using the *cp* value with the lowest cross-validated error:



In [ ]:
aux_cp_lowest_error <- aux_fit$cptable[aux_id_lowest_error, "CP"]
prune(aux_fit, cp = aux_cp_lowest_error)


### Bagging for regression trees

The following function automates the procedure of fitting and pruning a regression tree
for the median value:


In [ ]:
fitPrunedTree <- function(working_data) {
    working_fit <- rpart(medv ~ ., data = working_data)
    id_lowest_error <- which.min(working_fit$cptable[, "xerror"])
    cp_with_lowest_error <- working_fit$cptable[id_lowest_error, "CP"]
    prune(working_fit, cp = cp_with_lowest_error)
}


Now we can apply this procedure to all bootstrapped datasets:



In [ ]:
pruned_tree_list <- lapply(bootstrap_list, fitPrunedTree)
pruned_tree_list[1:5]


## Prediction

In the following code, we show how to perform predictions using all the fitted decision
trees for the bootstrapped datasets.

### Prepare the profile of interest

We perform the prediction for the first row.


In [ ]:
first_row <- Boston[1, ]
sprintf("Real median value: %d", first_row$medv)
first_row_without_median_value <- subset(first_row, select = -medv)
first_row_without_median_value


### Perform prediction

Predictions are made using each fitted regression tree:


In [ ]:
predictWithModel <- function(pruned_tree) {
    predict(pruned_tree, newdata = first_row_without_median_value)
}
predictions <- lapply(pruned_tree_list, predictWithModel)
predicted_values <- unlist(predictions)


Our best estimate is:



In [ ]:
sprintf("Predicted median value: %.*f", 2, mean(predicted_values))



Let's visualize the distribution of predictions and our best estimate:



In [ ]:
ggplot(data.frame(medv = predicted_values)) +
    geom_histogram(aes(x = medv), bins = 20, color = "black", fill = "lightblue") +
    geom_vline(aes(xintercept = medv, color = "Observed value"),
               data.frame(medv = first_row$medv), linetype = 1, linewidth = 0.8) +
    geom_vline(aes(xintercept = medv, color = "Predicted mean value"),
               data.frame(medv = mean(predicted_values)), linetype = 2, linewidth = 0.8) +
    labs(x = "Median home value", y = "Frequency", color = "")
